In [1]:
#18/05/21: stripped down version of IBM code for running locally on IBM Qsystem 1
import math
import certifi
# Importing standard Qiskit libraries and configuring account
import matplotlib.pyplot as plt
import numpy as np
from math import pi
from scipy import stats

from qiskit import QuantumCircuit, ClassicalRegister, QuantumRegister, execute, Aer,BasicAer, IBMQ
from qiskit.compiler import transpile, assemble
from qiskit.tools.jupyter import *
from qiskit.visualization import *
from qiskit.tools.visualization import circuit_drawer
from qiskit.quantum_info import state_fidelity

backend = BasicAer.get_backend('unitary_simulator')

IBMQ.enable_account('ebf27cd7dfdd3f9e5739d07c2894d6558560bd9a4d7bbf2e020744a946ce42caf7e7572b1b56aa2da9b0fc28082328bf0a4eb43541d046be4da501e0f341fc89')

provider = IBMQ.get_provider(hub='ibm-q-research')



/var/folders/8k/2jy31zh55w99h24zx25mm9980000gn/T/ipykernel_2531/3214329899.py:19: DeprecationWarning: The qiskit.IBMQ entrypoint and the qiskit-ibmq-provider package (accessible from 'qiskit.providers.ibmq`) are deprecated and will be removed in a future release. Instead you should use the qiskit-ibm-provider package which is accessible from 'qiskit_ibm_provider'. You can install it with 'pip install qiskit_ibm_provider'. Just replace 'qiskit.IBMQ' with 'qiskit_ibm_provider.IBMProvider'
  IBMQ.enable_account('ebf27cd7dfdd3f9e5739d07c2894d6558560bd9a4d7bbf2e020744a946ce42caf7e7572b1b56aa2da9b0fc28082328bf0a4eb43541d046be4da501e0f341fc89')


In [2]:
provider.backends()

[<IBMQSimulator('ibmq_qasm_simulator') from IBMQ(hub='ibm-q-research', group='anu-3', project='main')>,
 <IBMQSimulator('simulator_statevector') from IBMQ(hub='ibm-q-research', group='anu-3', project='main')>,
 <IBMQSimulator('simulator_mps') from IBMQ(hub='ibm-q-research', group='anu-3', project='main')>,
 <IBMQSimulator('simulator_extended_stabilizer') from IBMQ(hub='ibm-q-research', group='anu-3', project='main')>,
 <IBMQSimulator('simulator_stabilizer') from IBMQ(hub='ibm-q-research', group='anu-3', project='main')>,
 <IBMQBackend('ibm_lagos') from IBMQ(hub='ibm-q-research', group='anu-3', project='main')>,
 <IBMQBackend('ibm_nairobi') from IBMQ(hub='ibm-q-research', group='anu-3', project='main')>,
 <IBMQBackend('ibm_perth') from IBMQ(hub='ibm-q-research', group='anu-3', project='main')>]

In [3]:
#define Pauli matrices
s1 = np.matrix([[0,1],[1,0]])
s2 = np.matrix([[0,-1j],[1j,0]])
s3 = np.matrix([[1,0],[0,-1]])
Id=np.matrix([[1,0],[0,1]])

In [4]:
#define the states
def rhopos(v,alpha):
    rhoout=0.5*(Id+(1-v)*(s3*math.cos(alpha)+s1*math.sin(alpha)))
    
    return rhoout
    
def rhoneg(v,alpha):
    rhoout=0.5*(Id+(1-v)*(s3*math.cos(alpha)-s1*math.sin(alpha)))
    
    return rhoout
    

In [5]:
rhoneg(0.45,math.pi/4)

from numpy import linalg as LA
ang=math.pi/4*0.3



In [14]:
def circuit2qubit(zz1,zz2,quno1,quno2,xvec):
    
    qr=QuantumRegister(2)
    my_layout={qr[0]:quno1,qr[1]:quno2}
    #my_layout={qr[0]:quno1}
    circuit2 = QuantumCircuit();
    circuit2.add_register(qr)
    cr=ClassicalRegister(2)
    circuit2.add_register(cr)
    

    circuit2.initialize(zz1,0)
    circuit2.initialize(zz2,1)

    
    circuit2.u(xvec[0,0],xvec[0,1],xvec[0,2],1)
   
    circuit2.u(xvec[0,3],xvec[0,4],xvec[0,5],0)
    circuit2.cx(0,1)

    circuit2.rz(xvec[0,6],1)
    circuit2.ry(xvec[0,7],0)
    circuit2.cx(1,0)
    
    circuit2.ry(xvec[0,8],0)
    
    circuit2.cx(0,1)

    circuit2.u( xvec[0,9],xvec[0,10],xvec[0,11],1)
    circuit2.u(xvec[0,12],xvec[0,13],xvec[0,14],0)
    circuit2.measure(0,0);
    circuit2.measure(1,1);
    
    return circuit2

In [15]:
def totcirc2Qnoise(v,alpha,shotsin,quno1,quno2,xvec):
    shotseff=shotsin
    
    rho1=rhopos(v,alpha)
    rho2=rhoneg(v,alpha)
    
    #get eigenvalues and eigenvectors for preparing states in appropriate probabilities
    eigsyst1=LA.eigh(rho1)
    evals1=eigsyst1[0]
    evecs1=eigsyst1[1]
    
    eigsyst2=LA.eigh(rho2)
    evals2=eigsyst2[0]
    evecs2=eigsyst2[1]

        
    qc_list=[]
    shots_list=[]
    import random
    z11=[evecs1[0,0],evecs1[0,1]]
    z12=[evecs1[1,0],evecs1[1,1]]
    z21=[evecs2[0,0],evecs2[0,1]]
    z22=[evecs2[1,0],evecs2[1,1]]
 
    qr=QuantumRegister(2)
    #my_layout={qr[0]:q1list[0],qr[1]:q2list[0],qr[2]:q1list[1],qr[3]:q2list[1],qr[4]:q1list[2],qr[5]:q2list[2],qr[6]:q1list[3],qr[7]:q2list[3],qr[8]:q1list[4],qr[9]:q2list[4]}
    
    #state1 first
#     print(z11)
    circuit1state11=circuit2qubit(z11,z11,quno1,quno2,xvec)
    circuit1state12=circuit2qubit(z11,z12,quno1,quno2,xvec)
    circuit1state21=circuit2qubit(z12,z11,quno1,quno2,xvec)
    circuit1state22=circuit2qubit(z12,z12,quno1,quno2,xvec)

    
    #state2 first
    circuit2state11=circuit2qubit(z21,z21,quno1,quno2,xvec)
    circuit2state12=circuit2qubit(z21,z22,quno1,quno2,xvec)
    circuit2state21=circuit2qubit(z22,z21,quno1,quno2,xvec)
    circuit2state22=circuit2qubit(z22,z22,quno1,quno2,xvec)

    #result00 = execute(circuit00,backend=device,shots=p00+pthth+pth0+p0th).result()      
    qc_list.append(circuit1state11)
    shots_list.append(evals1[0]*evals1[0])
    qc_list.append(circuit1state12)
    shots_list.append(evals1[0]*evals1[1])
    qc_list.append(circuit1state21)
    shots_list.append(evals1[1]*evals1[0])
    qc_list.append(circuit1state22)
    shots_list.append(evals1[1]*evals1[1])
    
    
    qc_list.append(circuit2state11)
    shots_list.append(evals2[0]*evals2[0])
    qc_list.append(circuit2state12)
    shots_list.append(evals2[0]*evals2[1])
    qc_list.append(circuit2state21)
    shots_list.append(evals2[1]*evals2[0])
    qc_list.append(circuit2state22)
    shots_list.append(evals2[1]*evals2[1])



    return qc_list, shots_list

In [9]:
import qiskit.tools.jupyter
%qiskit_job_watcher

Accordion(children=(VBox(layout=Layout(max_width='710px', min_width='710px')),), layout=Layout(max_height='500…

<IPython.core.display.Javascript object>

In [10]:
from qiskit.tools import job_monitor

In [11]:
alphavec=np.linspace(math.pi/8+0.01,math.pi/8-0.01,3)
print(alphavec)
print(math.pi/8)

[0.40269908 0.39269908 0.38269908]
0.39269908169872414


In [12]:
import csv

results = []
with open('exported_data_v1_api6/v1Pn6Alppiover6_POVMs.txt') as csvfile:
    reader = csv.reader(csvfile, quoting=csv.QUOTE_NONNUMERIC) # change contents to floats
    for row in reader: # each row is a list
        results.append(row)
        
sp=np.shape(results)

POVM=[]
for kk in range(sp[0]):
    resultsint=results[kk]
    resultsint.pop(0)
    POVM.append(resultsint)
print(POVM)

Xvec= []
with open('exported_data_v1_api6/v1Pn6Alppiover6_Xvec.txt') as csvfile:
    reader = csv.reader(csvfile, quoting=csv.QUOTE_NONNUMERIC) # change contents to floats
    for row in reader: # each row is a list
        Xvec.append(row)
Xvec=np.array(Xvec)
print(Xvec)
print(np.size(Xvec))

[[0.179891773, 0.667214505, 0.377763753], [0.651152166, 0.0400741349, 0.519435781], [-0.54499616, 0.629339083, 0.0482712621], [-0.496607483, -0.396422989, 0.764951568]]
[[-1.74080887e+00 -1.69278285e-09  3.22954485e-08  8.68374399e-01
  -3.14159244e+00  1.13960570e-08  1.12128518e-08 -1.60183814e+00
   9.93854599e-01  3.26279370e+00 -1.54052844e-08  8.59829935e-09
   2.41054454e-01  3.14159212e+00  3.16817709e-07  1.11202755e-01]]
16


In [16]:
#loop over and repeat many times to see how often Bob's prediction is wrong based on Alices information
#alices measurement outcomes are the final qubit, which is why it might seem a little strange below

circuit_list=[]
prob_list=[]
shotstrain=100000
shotsrun=100000
    
nrep=11

#want to try 4,7 and 18,21
quno1=18;
quno2=17

alphavec=np.linspace(0.01,2*math.pi/4-0.01,nrep)
#alphavec=np.linspace(math.pi/6+0.01,math.pi/6-0.01,nrep)

v=0.08

for ii in range(nrep):
    #then repeat for 0 angle with the calibration from training data
    ain=alphavec[ii]
    print(ain)
    [circ_train,shots_out] =totcirc2Qnoise(v,ain,shotstrain,quno1,quno2,Xvec)
#     circuit_list.append(circ_train)
#     print(np.size(circuit_list))
#     print(circuit_list)
    for kk in range(8):
        circuit_list.append(circ_train[kk])
        prob_list.append(shots_out[kk])

#the probability list  only depends on alpha and v, so we only need to run this once

0.01
0.16507963267948966
0.3201592653589793
0.47523889803846897
0.6303185307179586
0.7853981633974483
0.9404777960769379
1.0955574287564276
1.2506370614359172
1.405716694115407
1.5607963267948965


In [17]:
#for each alpha value, we need to analyse the data for both states. Can plot out the individual probabilities and the probability of error

#need to load the correct probabilities above though corresponding to each alpha or v
folder1='pi_over_4_data/v08/'
plist1=[0.55,0.6,0.65,0.7,0.725,0.75,0.775,0.8,0.825,0.85,0.9,0.95]
folder2='pi_over_4_data/v1/'
plist2=[0.6,0.66,0.68,0.702,0.71,0.73,0.75,0.77,0.79,0.8,0.82,0.85,0.87]
folder3='pi_over_4_data/v12/'
plist3=[0.55,0.6,0.65,0.7,0.75,0.8,0.85,0.9]
folder4='pi_over_4_data/v15/'
plist4=[0.6,0.65,0.7,0.75,0.8,0.85,0.9]
#pi over 6 is v=0.08
folder5='pi_over_6_data/'
plist5=[0.55,0.6,0.65,0.7,0.75,0.8,0.85,0.9,0.95]

choice=4;

if choice==5:
    plist=plist5
    folder=folder5
    pinum=6
    strv='08'
if choice==4:
    plist=plist4
    folder=folder4
    pinum=4
    strv='15'
if choice==3:
    plist=plist3
    folder=folder3
    pinum=4
    strv='12'
if choice==2:
    plist=plist2
    folder=folder2
    pinum=4
    strv='1'
if choice==1:
    plist=plist1
    folder=folder1
    pinum=4
    strv='08'
    
cname='ibmq_ehningen'
#Prior probability for which this circuit was optimised

perr_save=[]

#save the appropriate counts for bootstrapping
count1_00_BS=[]
count1_10_BS=[]
count1_01_BS=[]
count1_11_BS=[]
count2_00_BS=[]
count2_10_BS=[]
count2_01_BS=[]
count2_11_BS=[]
for kk in range(len(plist)):
    Pn=plist[kk]
    st=str(Pn)
    sliced = st[2:]
    
    stv=str(v)
    sliced2 = stv[2:]
    
    data_load = np.load(str(folder+cname+str("_LR_2022_disc_q1817_Api")+str(pinum)+str("_v")+strv+str("_PN")+sliced+str("_long")+str("/")+str("data")+str("/")+str("all_data"))+str(".npz"),allow_pickle=True)
    data=data_load['arr_0']


    count=0

    count1_00_fin=[]
    count1_10_fin=[]
    count1_01_fin=[]
    count1_11_fin=[]
    count2_00_fin=[]
    count2_10_fin=[]
    count2_01_fin=[]
    count2_11_fin=[]

    totcount1=[]
    totcount2=[]

    #loop over the alpha values
    for kk in range(nrep):

        #loop over the different measurement results
        count1_00=0
        count1_01=0
        count1_10=0
        count1_11=0

        count2_00=0
        count2_01=0
        count2_10=0
        count2_11=0
        countint=0
        for ii in range(8):
            if countint<4:
                data1=data[count]
                prob1=prob_list[count]

        #         print(prob1)
                #probability of getting 00 having sent state 1
                count1_00=count1_00+prob1*data1.get('00',0)
                count1_01=count1_01+prob1*data1.get('01',0)
                count1_10=count1_10+prob1*data1.get('10',0)
                count1_11=count1_11+prob1*data1.get('11',0)
            else:

                data2=data[count]
                prob2=prob_list[count]
                count2_00=count2_00+prob2*data2.get('00',0)
                count2_01=count2_01+prob2*data2.get('01',0)
                count2_10=count2_10+prob2*data2.get('10',0)
                count2_11=count2_11+prob2*data2.get('11',0)

            count=count+1
            countint=countint+1

        count1_00_fin.append(count1_00)
        count1_01_fin.append(count1_01)
        count1_10_fin.append(count1_10)
        count1_11_fin.append(count1_11)

        totcount1.append(count1_00+count1_01+count1_10+count1_11)
        totcount2.append(count2_00+count2_01+count2_10+count2_11)

        count2_00_fin.append(count2_00)
        count2_01_fin.append(count2_01)
        count2_10_fin.append(count2_10)
        count2_11_fin.append(count2_11)


#     plt.plot(count1_00_fin)
#     plt.plot(count2_00_fin)

#     plt.figure(2)


#     plt.plot(count1_10_fin)
#     plt.plot(count2_10_fin)

#     plt.figure(3)

#     plt.plot(count1_01_fin)
#     plt.plot(count2_01_fin)
#     plt.figure(4)
#     plt.plot(count1_11_fin)
#     plt.plot(count2_11_fin)

    count1_00_BS.append(count1_00_fin[5])
    count1_01_BS.append(count1_01_fin[5])
    count1_10_BS.append(count1_10_fin[5])
    count1_11_BS.append(count1_11_fin[5])
    
    count2_00_BS.append(count2_00_fin[5])
    count2_01_BS.append(count2_01_fin[5])
    count2_10_BS.append(count2_10_fin[5])
    count2_11_BS.append(count2_11_fin[5])
    #now compute the error probability for each state
    Perr=[]
    for jjj in range(nrep):
        totcounts1=count1_00_fin[jjj]+count1_01_fin[jjj]+count1_10_fin[jjj]+count1_11_fin[jjj]
        totcounts2=count2_00_fin[jjj]+count2_01_fin[jjj]+count2_10_fin[jjj]+count2_11_fin[jjj]
        #Perr.append(Pn*(count1_01_fin[kk]+count1_10_fin[kk]+count1_11_fin[kk])/totcounts1+(1-Pn)*(count2_00_fin[kk])/totcounts2)
        Perr.append(Pn*(count1_00_fin[jjj])/totcounts1+(1-Pn)*(count2_01_fin[jjj]+count2_10_fin[jjj]+count2_11_fin[jjj])/totcounts2)

#     print(Perr)
    perr_save.append(Perr[5])
#expected for v=0.02, Pn=0.95 is 0.0184, Anything below 0.02 should be quantum adv
perr_save

print(totcounts1,totcounts2)

100000.0 100000.00000000001


In [18]:
print(perr_save)

[0.1236229152, 0.12337892, 0.10684392480000002, 0.09656803200000001, 0.0836429504, 0.0722273736, 0.05944822239999999]


In [28]:
#might not need error mitigation, but we should bootstrap here. And automate the whole thing
#bootstrapping based on the counts in the previous run^
meanerr_save=[]
errbar_save=[]
ntrials=1000
for kk in range(len(plist)):
    #prior probability of each state
    Pn=plist[kk]
    #load the counts when states 1 and 2 are prepared
    count1_00=count1_00_BS[kk]
    count1_01=count1_01_BS[kk]
    count1_10=count1_10_BS[kk]
    count1_11=count1_11_BS[kk]
    #probabilities
    p1_00=count1_00/shotsrun
    p1_01=count1_01/shotsrun
    p1_10=count1_10/shotsrun
    p1_11=count1_11/shotsrun
    
    count2_00=count2_00_BS[kk]
    count2_01=count2_01_BS[kk]
    count2_10=count2_10_BS[kk]
    count2_11=count2_11_BS[kk]
    #probabilities
    p2_00=count2_00/shotsrun
    p2_01=count2_01/shotsrun
    p2_10=count2_10/shotsrun
    p2_11=count2_11/shotsrun
    
    Perr=[]
    #then sample from this probability distribution many times
    
    for kk in range(ntrials):
        rv1= np.random.multinomial(shotsrun, [p1_00,p1_01,p1_10,p1_11], size=1)
        n1_00=rv1[0][0]
        n1_01=rv1[0][1]
        n1_10=rv1[0][2]
        n1_11=rv1[0][3]
        
        rv2= np.random.multinomial(shotsrun, [p2_00,p2_01,p2_10,p2_11], size=1)
        n2_00=rv2[0][0]
        n2_01=rv2[0][1]
        n2_10=rv2[0][2]
        n2_11=rv2[0][3]
       
        #make a probability distribution and sample from it every time
        totcounts1=n1_00+n1_01+n1_10+n1_11
        totcounts2=n1_00+n1_01+n1_10+n1_11
        #Perr.append(Pn*(count1_01_fin[kk]+count1_10_fin[kk]+count1_11_fin[kk])/totcounts1+(1-Pn)*(count2_00_fin[kk])/totcounts2)
        Perr.append(Pn*(n1_00)/totcounts1+(1-Pn)*(n2_01+n2_10+n2_11)/totcounts2)


    errbar_save.append(np.std(Perr))
    meanerr_save.append(np.mean(Perr))
    
print(errbar_save)
print(meanerr_save)

[0.0007337906415386819, 0.0007017742068756874, 0.0006401369890820242, 0.0006507611384210031, 0.0006219350251894784, 0.0005642002136597877, 0.0005718215025423141, 0.0005948459218016043, 0.0005600774303252637, 0.0005510725513387505, 0.00047098376709924937, 0.0004044898404654312]
[0.1277700405, 0.118450742, 0.110009953, 0.104601061, 0.10044693975000002, 0.09437298749999999, 0.0899435445, 0.08295764599999998, 0.0839981845, 0.07443088100000002, 0.056598377, 0.03947988950000001]
